$$Module Overview$$

Module Name: non_parametric_fixed_income.py

Purpose:
Provide classes and methods to compute non‑parametric risk measures such as Value at Risk (VaR) and Expected Shortfall (ES) using quantile, bootstrap, and Extreme Value Theory (EVT) approaches.

**Class: FixedIncomeNprmSingle**

This class computes VaR and ES for a single fixed‑income security based on historical returns.

__init__(self, returns, position, alpha=0.05, method="quantile", n_bootstrap_samples=10000)

In [ ]:
"""
Initialize a FixedIncomeNprmSingle object.

Parameters:
    returns (np.ndarray): 
        Historical returns of the fixed-income security (1D numeric numpy array).
    position (float): 
        Quantity held (positive for long positions, negative for short positions).
    alpha (float, optional): 
        Significance level for VaR and ES (e.g., 0.05). Default is 0.05.
    method (str, optional): 
        The calculation method to use, either "quantile" or "bootstrap". Default is "quantile".
    n_bootstrap_samples (int, optional): 
        Number of bootstrap samples to use if method is "bootstrap". Default is 10000.

Attributes:
    returns (np.ndarray): Validated returns array.
    position (float): The position size.
    alpha (float): The significance level.
    method (str): The chosen method ("quantile" or "bootstrap").
    n_bootstrap_samples (int): Number of bootstrap iterations (if applicable).
    var (float): Computed Value at Risk.
    es (float): Computed Expected Shortfall.

Automatically calls:
    fit() - to compute VaR and ES upon instantiation.
"""

fit(self)

In [ ]:
"""
Compute VaR and ES using the specified method.

If method == "quantile":
    Uses a quantile-based approach:
        VaR = -position * Q_alpha(returns)
        ES  = -position * E[returns | returns < Q_alpha(returns)] (for long positions)
        (or the corresponding formula for short positions using Q_(1-alpha)).
If method == "bootstrap":
    Uses a bootstrap resampling approach to compute VaR and ES.

Updates:
    self.var and self.es with the computed risk measures.
"""

evt(self, quantile_threshold=0.95) -> dict

In [ ]:
"""
Estimate VaR and ES using Extreme Value Theory (GPD).

Procedure:
1. Convert returns to losses:
    - For long positions: losses = -returns
    - For short positions: losses = returns
2. Set threshold u as the quantile_threshold (e.g., 0.95) of the losses.
3. Compute exceedances: exceedances = losses[losses > u] - u.
4. Estimate GPD parameters (xi and beta) using maximum likelihood (minimize the negative log-likelihood).
5. Compute EVT VaR and ES using:
    VaR_evt = u + (beta/xi)*(((n/n_u)*(1-alpha))^(-xi) - 1)
    ES_evt = (VaR_evt + beta - xi*u)/(1 - xi)
6. Scale the final results by |position|.

Parameters:
    quantile_threshold (float, optional): 
        The quantile to use for setting the threshold u (default is 0.95).

Returns:
    dict: Contains keys:
        - "evt_var": EVT-based VaR (float)
        - "evt_es": EVT-based ES (float)
        - "xi": Estimated shape parameter (float)
        - "beta": Estimated scale parameter (float)
        - "u": The threshold used (float)
        - "n": Total number of observations (int)
        - "n_u": Number of exceedances (int)
"""

summary(self) -> dict

In [ ]:
"""
Return a summary of key risk metrics.

For a long position:
    maxLoss = -position * min(returns)
For a short position:
    maxLoss = -position * max(returns)

Other metrics:
    maxExcessLoss = maxLoss - var
    esOverVar = es / var (if var != 0)
    maxExcessLossOverVar = maxExcessLoss / var (if var != 0)

Returns:
    dict: Dictionary containing:
        - "var": Computed Value at Risk (float)
        - "es": Computed Expected Shortfall (float)
        - "maxLoss": Maximum loss over the sample period (float)
        - "maxExcessLoss": Difference between maxLoss and VaR (float)
        - "maxExcessLossOverVar": Ratio of maxExcessLoss to VaR (float or None)
        - "esOverVar": Ratio of ES to VaR (float or None)
"""

**Class: FixedIncomeNprmPort**

This class computes risk measures for a portfolio of fixed‑income positions by first aggregating the returns.

__init__(self, returns, positions, alpha=0.05, method="quantile", n_bootstrap_samples=10000)


In [ ]:
"""
Initialize a FixedIncomeNprmPort object for a portfolio.

Parameters:
    returns (np.ndarray):
        2D array of historical returns (rows = periods, columns = securities).
    positions (list of float):
        List of positions for each security (must match the number of columns in returns).
    alpha (float, optional):
        Significance level for VaR/ES (default is 0.05).
    method (str, optional):
        Calculation method ("quantile" or "bootstrap"). Default is "quantile".
    n_bootstrap_samples (int, optional):
        Number of bootstrap samples if using the bootstrap method. Default is 10000.

Attributes:
    returns (np.ndarray): The 2D returns matrix.
    positions (list): The list of positions.
    alpha (float): Significance level.
    method (str): Calculation method.
    n_bootstrap_samples (int): Bootstrap sample count.
    portfolio_returns (np.ndarray):
        Aggregated portfolio returns computed as the weighted sum of each period's returns.
    cum_returns (np.ndarray):
        Cumulative compounded portfolio returns.
    var (float): Computed portfolio VaR.
    es (float): Computed portfolio ES.

Automatically calls:
    fit() - to compute risk measures upon instantiation.
"""

fit(self)

In [ ]:
"""
Compute the portfolio risk measures.

Procedure:
1. Aggregate the returns:
       portfolio_returns[t] = sum_i( positions[i] * returns[t,i] )
2. Compute cumulative returns:
       cum_returns[t] = product_{k=1..t}(1 + portfolio_returns[k]) - 1
3. Based on the chosen method ("quantile" or "bootstrap"), compute portfolio VaR and ES.
    - For a net long portfolio, use the α-quantile.
    - For a net short portfolio, use the (1-α)-quantile.

Updates:
    self.portfolio_returns, self.cum_returns, self.var, and self.es.
"""

marg_vars(self, scale_factor=0.1) -> list

In [ ]:
"""
Compute the Marginal VaR for each individual position in the portfolio.

The marginal VaR for position i is approximated as:
    MVaR_i ≈ (VaR_new - VaR_original) / scale_factor
where VaR_new is computed after perturbing the i-th position by scale_factor.

Parameters:
    scale_factor (float, optional):
        The small increment used to perturb each position (default is 0.1).

Returns:
    list of float:
        A list of marginal VaR values corresponding to each position.
        
After computing, the original portfolio risk measures are restored.
"""

summary(self) -> dict

In [ ]:
"""
Return a summary dictionary of key portfolio risk metrics.

Procedure:
- Compute the net portfolio returns and determine maxLoss:
    For a net long portfolio: maxLoss = -sum(positions) * min(portfolio_returns)
    For a net short portfolio: maxLoss = -sum(positions) * max(portfolio_returns)
- Then compute:
    maxExcessLoss = maxLoss - var
    esOverVar = es / var if var != 0
    maxExcessLossOverVar = maxExcessLoss / var if var != 0

Returns:
    dict:
        {
            "var": Computed portfolio VaR (float),
            "es": Computed portfolio ES (float),
            "maxLoss": Maximum loss (float),
            "maxExcessLoss": Difference between maxLoss and VaR (float),
            "maxExcessLossOverVar": Ratio (float or None),
            "esOverVar": Ratio (float or None)
        }
"""